# Webscraper FutureLearn
FeatureLearn.com is an online learning platform, offering courses on a variety of subjects. This scraper scrapes all the courses available on futurelearn. The total run time is around an hour and a half.

## 1 Importing packages

In [4]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import csv

## 2 Preparation

### 2.1 Getting the categories and category numbers
Although these could be collected manually too, the category numbers are quite arbitrarly and some are hidden and get redirected to the 'all courses' category. Hence, the category numbers were collected manually.

In [3]:
categories_numbers = [{'name': 'Business & Management', 'category_number': 8},
 {'name': 'Creative Arts & Media', 'category_number': 9},
 {'name': 'Nature & Environment', 'category_number': 10},
 {'name': 'Politics & Society', 'category_number': 11},
 {'name': 'Literature', 'category_number': 12},
 {'name': 'Healthcare & Medicine', 'category_number': 13},
 {'name': 'Science, Engineering & Maths', 'category_number': 14},
 {'name': 'Law', 'category_number': 15},
 {'name': 'History', 'category_number': 16},
 {'name': 'IT & Computer Science', 'category_number': 17},
 {'name': 'Teaching', 'category_number': 18},
 {'name': 'Language', 'category_number': 19},
 {'name': 'Study Skills', 'category_number': 21},
 {'name': 'Psychology & Mental Health', 'category_number': 22}]

### 2.2 Getting the max page number
To retrieve all the page numbers, we must first determine what the maximum page is per category.

In [6]:
def look_for_max_page_per_category(categories_numbers):    
    for category_number in categories_numbers:
        url = f"https://www.futurelearn.com/courses?filter_category={category_number['category_number']}&filter_course_type=open&filter_availability=started"
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        category_number['max_pages'] = int(soup.find('div', class_ = "a-content a-contiguous-top u-centered a-content--tight").find_all('li', class_="pagination-module_item__3XB-l")[-1].text)
        time.sleep(5)
    max_pages_per_category = categories_numbers
    return max_pages_per_category

In [7]:
max_pages_per_category = look_for_max_page_per_category(categories_numbers)
max_pages_per_category

[{'name': 'Business & Management', 'category_number': 8, 'max_pages': 30},
 {'name': 'Creative Arts & Media', 'category_number': 9, 'max_pages': 10},
 {'name': 'Nature & Environment', 'category_number': 10, 'max_pages': 9},
 {'name': 'Politics & Society', 'category_number': 11, 'max_pages': 11},
 {'name': 'Literature', 'category_number': 12, 'max_pages': 2},
 {'name': 'Healthcare & Medicine', 'category_number': 13, 'max_pages': 19},
 {'name': 'Science, Engineering & Maths',
  'category_number': 14,
  'max_pages': 15},
 {'name': 'Law', 'category_number': 15, 'max_pages': 3},
 {'name': 'History', 'category_number': 16, 'max_pages': 5},
 {'name': 'IT & Computer Science', 'category_number': 17, 'max_pages': 16},
 {'name': 'Teaching', 'category_number': 18, 'max_pages': 13},
 {'name': 'Language', 'category_number': 19, 'max_pages': 6},
 {'name': 'Study Skills', 'category_number': 21, 'max_pages': 5},
 {'name': 'Psychology & Mental Health', 'category_number': 22, 'max_pages': 8}]

### 2.3 Make the page urls
This code does not require a html request since each link is predictable. The resulting dictionary contains the category name, page number and page url.

In [311]:
 def make_page_urls(max_pages_per_category):   
    page_urls = []
    for category in categories:
        for page in range(1, category['max_pages']+1):
            page_info = {}
            page_info['category'] = category['name']
            page_info['page'] = page
            page_info['page_url'] = f"https://www.futurelearn.com/courses?filter_category={category['category_number']}&filter_course_type=open&filter_availability=started&page={page}#courses-grid-start"
            page_urls.append(page_info)
    return page_urls

In [312]:
page_urls = make_page_urls(max_pages_per_category)
page_urls

[{'category': 'Business & Management',
  'page': 1,
  'page_url': 'https://www.futurelearn.com/courses?filter_category=8&filter_course_type=open&filter_availability=started&page=1#courses-grid-start'},
 {'category': 'Business & Management',
  'page': 2,
  'page_url': 'https://www.futurelearn.com/courses?filter_category=8&filter_course_type=open&filter_availability=started&page=2#courses-grid-start'},
 {'category': 'Business & Management',
  'page': 3,
  'page_url': 'https://www.futurelearn.com/courses?filter_category=8&filter_course_type=open&filter_availability=started&page=3#courses-grid-start'},
 {'category': 'Business & Management',
  'page': 4,
  'page_url': 'https://www.futurelearn.com/courses?filter_category=8&filter_course_type=open&filter_availability=started&page=4#courses-grid-start'},
 {'category': 'Business & Management',
  'page': 5,
  'page_url': 'https://www.futurelearn.com/courses?filter_category=8&filter_course_type=open&filter_availability=started&page=5#courses-grid

### 2.4 Getting the course urls
Each page contains up to 15 courses. The following code retrieves these data and makes a csv file for later use. *Ik kon niet direct de csv schrijven omdat ik een gekke TypeError kreeg*

In [396]:
def make_course_urls(page_urls):
    course_urls = []
    #f = open("future_learn_page_urls.csv", "w", encoding = "UTF-8")
    #writer = csv.writer(f, delimiter = ";")
    #writer.writerow(['course_url', 'page', 'category']
    list_of_inaccessible_pages = []                
    for page_url in page_urls:
        try:
            r = requests.get(page_url['page_url'])
            soup = BeautifulSoup(r.text, "html.parser")
            courses = soup.find(class_="cardGrid-wrapper_2TvtF cardGrid-hasSideNav_1sLqj").find_all('div', class_="m-card Container-wrapper_1lZbP Container-grey_1l9VP")
            for course in courses:
                course_url = {}
                course_url['course_url'] = course.find_all('a')[0]['href']
                course_url['page'] = page_url['page']
                course_url['category'] = page_url['category']
                course_urls.append(course_url)
                #writer.writerow([course_url['course_url'], course_url['page'], course_url['category']]
        except:
            list_of_inaccessible_pages.append(page_url)
                
        time.sleep(1)
        print(f"Currently scraping page {page_url}")
    print(list_of_inaccessible_pages)
    print(f"Number of unaccessible pages: {len(list_of_inaccessible_pages)}")
    #f.close()
    return course_urls

In [378]:
course_urls = make_course_urls(page_urls)
course_urls

Currently scraping page {'category': 'Business & Management', 'page': 1, 'page_url': 'https://www.futurelearn.com/courses?filter_category=8&filter_course_type=open&filter_availability=started&page=1#courses-grid-start'}
Currently scraping page {'category': 'Business & Management', 'page': 2, 'page_url': 'https://www.futurelearn.com/courses?filter_category=8&filter_course_type=open&filter_availability=started&page=2#courses-grid-start'}
Currently scraping page {'category': 'Business & Management', 'page': 3, 'page_url': 'https://www.futurelearn.com/courses?filter_category=8&filter_course_type=open&filter_availability=started&page=3#courses-grid-start'}
Currently scraping page {'category': 'Business & Management', 'page': 4, 'page_url': 'https://www.futurelearn.com/courses?filter_category=8&filter_course_type=open&filter_availability=started&page=4#courses-grid-start'}
Currently scraping page {'category': 'Business & Management', 'page': 5, 'page_url': 'https://www.futurelearn.com/course

Currently scraping page {'category': 'Creative Arts & Media', 'page': 9, 'page_url': 'https://www.futurelearn.com/courses?filter_category=9&filter_course_type=open&filter_availability=started&page=9#courses-grid-start'}
Currently scraping page {'category': 'Creative Arts & Media', 'page': 10, 'page_url': 'https://www.futurelearn.com/courses?filter_category=9&filter_course_type=open&filter_availability=started&page=10#courses-grid-start'}
Currently scraping page {'category': 'Nature & Environment', 'page': 1, 'page_url': 'https://www.futurelearn.com/courses?filter_category=10&filter_course_type=open&filter_availability=started&page=1#courses-grid-start'}
Currently scraping page {'category': 'Nature & Environment', 'page': 2, 'page_url': 'https://www.futurelearn.com/courses?filter_category=10&filter_course_type=open&filter_availability=started&page=2#courses-grid-start'}
Currently scraping page {'category': 'Nature & Environment', 'page': 3, 'page_url': 'https://www.futurelearn.com/cours

Currently scraping page {'category': 'Healthcare & Medicine', 'page': 15, 'page_url': 'https://www.futurelearn.com/courses?filter_category=13&filter_course_type=open&filter_availability=started&page=15#courses-grid-start'}
Currently scraping page {'category': 'Healthcare & Medicine', 'page': 16, 'page_url': 'https://www.futurelearn.com/courses?filter_category=13&filter_course_type=open&filter_availability=started&page=16#courses-grid-start'}
Currently scraping page {'category': 'Healthcare & Medicine', 'page': 17, 'page_url': 'https://www.futurelearn.com/courses?filter_category=13&filter_course_type=open&filter_availability=started&page=17#courses-grid-start'}
Currently scraping page {'category': 'Healthcare & Medicine', 'page': 18, 'page_url': 'https://www.futurelearn.com/courses?filter_category=13&filter_course_type=open&filter_availability=started&page=18#courses-grid-start'}
Currently scraping page {'category': 'Healthcare & Medicine', 'page': 19, 'page_url': 'https://www.futurelea

Currently scraping page {'category': 'IT & Computer Science', 'page': 11, 'page_url': 'https://www.futurelearn.com/courses?filter_category=17&filter_course_type=open&filter_availability=started&page=11#courses-grid-start'}
Currently scraping page {'category': 'IT & Computer Science', 'page': 12, 'page_url': 'https://www.futurelearn.com/courses?filter_category=17&filter_course_type=open&filter_availability=started&page=12#courses-grid-start'}
Currently scraping page {'category': 'IT & Computer Science', 'page': 13, 'page_url': 'https://www.futurelearn.com/courses?filter_category=17&filter_course_type=open&filter_availability=started&page=13#courses-grid-start'}
Currently scraping page {'category': 'IT & Computer Science', 'page': 14, 'page_url': 'https://www.futurelearn.com/courses?filter_category=17&filter_course_type=open&filter_availability=started&page=14#courses-grid-start'}
Currently scraping page {'category': 'IT & Computer Science', 'page': 15, 'page_url': 'https://www.futurelea

[{'course_url': '/courses/sustainable-business',
  'page': 1,
  'category': 'Business & Management'},
 {'course_url': '/courses/omni-channel-retailing',
  'page': 1,
  'category': 'Business & Management'},
 {'course_url': '/courses/introduction-to-digital-marketing-digital-marketing-in-perspective',
  'page': 1,
  'category': 'Business & Management'},
 {'course_url': '/courses/introduction-to-digital-marketing-content-marketing',
  'page': 1,
  'category': 'Business & Management'},
 {'course_url': '/courses/introduction-to-digital-marketing-marketing-channel-objectives',
  'page': 1,
  'category': 'Business & Management'},
 {'course_url': '/courses/introduction-to-digital-marketing-personas-and-user-journeys',
  'page': 1,
  'category': 'Business & Management'},
 {'course_url': '/courses/edward-jenner-programme-course-0-an-introduction-to-personal-development',
  'page': 1,
  'category': 'Business & Management'},
 {'course_url': '/courses/advanced-good-pharmacy-practice-part1',
  'page

In [389]:
with open("../../gen/input/futurelearn_course_urls.csv", "w", encoding = "UTF-8") as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(['course_url', 'page', 'category'])
    for course_url in course_urls:
        writer.writerow([course_url['course_url'], course_url['page'], course_url['category']])

## 3 Data Collection

### 3.1 Getting the course info
The following function retrieves the following data on each page:
- Url: The complete url of the course
- Time: The exact time the data were scraped
- Page: The page from the category
- Header: The header of the course
- Enrollment: The number of enrollments of the course. Note that new courses do not have the information yet
- New: Dummy showing whether the course is new
- Review_count: Variable showing how many reviews there are
- Description: Short description of the course
- Duration: Duration of the course in weeks
- Weekly_study: Study time required on a weekly bases
- Unlimited: Dummy showing whether you can access this course with a so-called "unlimited" subscription. Not_present means that's not the case
- 100_online: Dummy showing whether the course is for 100% online
- Free: Dummy showing whether the course is for free. Not_present meaning it's not free
- Part_of_expert: Dummy showing whether the course is part of a larger expert course. Not_present means it's not the case
- Name_school: The name of the school that teaches the course
- Endorsed: Dummy showing whether the course is endorsed by third parties. 

Although the course page contained more information (e.g. in-depth course descriptions, numerous teachers), the variables above are relevant for an analysis. The sleep time is 1 second per retrieval

In [393]:
def get_course_info(course_url):
    course_info = {}
    complete_url = f"https://www.futurelearn.com{course_url['course_url']}"
    r = requests.get(complete_url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    #META-DATA
    ##url
    course_info['url'] = complete_url
    
    ##time
    course_info['Time'] = datetime.now()
    
    ##page
    course_info['page'] = course_url['page']
    
    ##category
    course_info['category'] = course_url['category']
    
    #HEADER
    try:
        course_info['Header'] = soup.find('h1').text
    except:
        course_info['Header'] = ''
        
    #NUMBER OF ENROLLMENTS
    try:
        course_info['Enrollments'] = soup.find('div', class_="spacer-module_default__3N2H9 spacer-module_vertical-4__5ZLo8").find('p', class_ = "text-module_wrapper__FfvIV text-module_black__2u5Rt text-module_sBreakpointSizexsmall__2Jlmd text-module_sBreakpointAlignmentleft__1MvbB text-module_isRegular__1K97K").text
    except:
        course_info['Enrollments'] = ''
    
    #NEW COURSE
    try:
        if soup.find('span', class_ = "Ribbon-module_wrapper__312EV Ribbon-module_coral__B1J53 Ribbon-module_isUppercase__1NTSw").text == 'New':
            course_info['New'] = 'yes'
        else:
            course_info['New'] = 'no' 
    except:
        course_info['New'] = 'no'

    #STAR RATING
    try:
        course_info['star_rating'] = soup.find(class_="spacer-module_default__3N2H9 spacer-module_left-1__1AJxh").text.split()[0]
    except:
        course_info['star_rating'] = ''
    
    #NUM OF REVIEWS
    try:
        course_info['review_count'] = soup.find('div', class_="ReviewStars-text_mSEFD").find('span').text
    except:
        course_info['review_count'] = ''
        
    #DESCRIPTION
    try:
        course_info['description'] = soup.find('div', class_ = "stack-module_wrapper__3ZERF").find(class_="text-module_wrapper__FfvIV text-module_black__2u5Rt text-module_sBreakpointSizemedium__2qitW text-module_mBreakpointSizemedium__1_OnK text-module_lBreakpointSizemedium__1Yq39 text-module_xlBreakpointSizemedium__1nNCx text-module_xxlBreakpointSizelarge__1uhrp text-module_sBreakpointAlignmentleft__1MvbB text-module_isRegular__1K97K").text
    except:
        course_info['description'] = ''
    
    #INFO COURSE
    try:
        for li in soup.find('div', class_ = "PageHeader-keyInfoWrapper_39HpT").find_all(class_="keyInfo-module_itemText__3w63w"):
            ## Top part of cell
            table_header = li.find(class_ = "text-module_wrapper__FfvIV text-module_mediumGrey__1uvOt text-module_sBreakpointSizesmall__3K4b4 text-module_sBreakpointAlignmentleft__1MvbB text-module_isInline__m5cFK text-module_isRegular__1K97K").text
            ## Lower part of cell
            table_text = li.find(class_ = "keyInfo-module_content__1K_85").text
            if table_header == 'Duration':
                course_info['Duration'] = table_text
            if table_header == 'Weekly study':
                course_info['Weekly_study'] = table_text
            if table_header == 'Unlimited':
                course_info['Unlimited'] = 'yes'
            if table_header == '100% online':
                course_info['100_online'] = 'yes'
            if table_header == 'Digital upgrade':
                course_info['Free'] = 'yes'
            if table_header == 'Accreditation':
                course_info['Accreditation'] = table_text
            if table_header == "Included in an ExpertTrack":
                course_info['Part_of_Expert'] = 'yes'
        table_headers_options = ['Duration', 'Weekly_study', 'Unlimited', '100_online', 'Free', 'Accreditation', 'Part_of_Expert']
        ##Looks whether a cell is missing
        for table_headers_option in table_headers_options:
            if table_headers_option not in course_info:
                course_info[table_headers_option] = 'Not_present'
    ##If there's an error in loading the table, place empty values
    except:
        for table_headers_option in table_headers_options:
            course_info[table_headers_option] = ''
    
    #NAME SCHOOL
    try:
        course_info['Name_school'] = soup.find('h2', class_="heading-module_wrapper__2dcxt heading-module_sBreakpointAlignmentleft__pCA_Y heading-module_sBreakpointSizelarge__SiUxO heading-module_black__Uge9G heading-module_isRegular__2NZyV").text
    except:
        course_info['Name_school'] = ''
    print(f"Currently scraping: {course_info['Header']}")
    
    #ENDORSERS (Check how often present)
    try:
        if soup.find('h2', class_ = "heading-module_wrapper__2dcxt heading-module_sBreakpointAlignmentcenter__2R3nG heading-module_sBreakpointSizelarge__SiUxO heading-module_black__Uge9G heading-module_isRegular__2NZyV").text == 'Endorsers and supporters':
            course_info['Endorsed'] = 'yes'
        else:
            course_info['Endorsed'] = 'no'
    except:
        course_info['Endorsed'] = 'no'
    time.sleep(1)
    return course_info

### 3.2 Scraping each course page and saving it in csv
The following code uses the function above to retrieve the relevant data of each page and stores it into a csv file. Note the delimiter is a ';'. The length of the list of inaccessable courses was 0 during the last run. It took 1 hour and 7 minutes to complete.

In [3]:
def write_data(course_urls):
    courses_info = []
    list_of_inaccessable_courses = []
    counter = 1
    with open("../../gen/output/futurelearn_data.csv", "w", encoding = "UTF-8") as csv_file:
        writer = csv.writer(csv_file, delimiter = ";")
        writer.writerow(['Url', 'Time', 'Page', 'Category', 'Header', 'Enrollments', 'New', 'Star_rating', 'Review_count', 'Description', 'Duration', 'Weekly_study', 'Unlimited', '100_online', 'Free', 'Accreditation', 'Part_of_Expert', 'Name_school', 'Endorsed'])
        for course_url in course_urls:
            try:
                course_info = get_course_info(course_url)
                courses_info.append(course_info)
                writer.writerow([course_info['url'], course_info['Time'], course_info['page'], course_info['category'], course_info['Header'], course_info['Enrollments'], course_info['New'], course_info['star_rating'], course_info['review_count'], course_info['description'], course_info['Duration'], course_info['Weekly_study'], course_info['Unlimited'], course_info['100_online'], course_info['Free'], course_info['Accreditation'], course_info['Part_of_Expert'], course_info['Name_school'], course_info['Endorsed']])
            except:
                list_of_inaccessable_courses.append(course_url)
            print(f'Processing... [{counter}/{len(course_urls)}]')
            counter += 1
    print(list_of_inaccessable_courses)
    print(f"Couldn't access {len(list_of_inaccessable_courses)} courses")
    return courses_info

In [398]:
start_time = datetime.now()
data = write_data(course_urls)
end_time = datetime.now()
print("Time it took to scrape: " + str(end_time - start_time))

Currently scraping: Business Futures: Sustainable Business Through Green HR
Currently scraping: Business Futures: Understanding Omni-channel Retailing and Supply Chains
Currently scraping: Digital Marketing in Perspective
Currently scraping: Digital Marketing: Content Marketing
Currently scraping: Digital Marketing: Marketing Channel Objectives
Currently scraping: Digital Marketing: Personas and User Journeys
Currently scraping: Edward Jenner: 0 - An Introduction to Personal Development
Currently scraping: Good Pharmacy Practice: Medication Management
Currently scraping: Social Media in Healthcare: Opportunities and Challenges
Currently scraping: Digital Skills: Digital Marketing
Currently scraping: Digital Skills: Digital Skills for Work and Life
Currently scraping: Digital Skills: Mobile
Currently scraping: Digital Skills: Social Media
Currently scraping: Digital Skills: Web Analytics
Currently scraping: Electrification of Urban Mobility: How to Get it Right
Currently scraping: How t

Currently scraping: Personal Trainer's Toolkit: Mental Health First Aid for Fitness Trainers
Currently scraping: Personal Trainer's Toolkit: Providing Nutritional Advice to Your Clients
Currently scraping: Personal Trainer's Toolkit: Workplace Health and Safety in a Fitness Setting
Currently scraping: Successful Negotiation: Essential Strategies and Skills
Currently scraping: The Science of Success: What Researchers Know that You Should Know
Currently scraping: Understanding Fashion: From Business to Culture
Currently scraping: Additive Manufacturing for Business: Practices and Ecosystem
Currently scraping: Career Management for Early Career Academic Researchers
Currently scraping: Decentralised Finance: Blockchain, Ethereum, and The Future of Banking
Currently scraping: Placemaking and Public Space Design: Unlocking Tourist Destinations
Currently scraping: Why Research Matters
Currently scraping: Fundamentals of Digital Marketing
Currently scraping: Product Management Essentials
Curre

Currently scraping: Business Etiquette: Master Communication and Soft Skills
Currently scraping: Collaborative Working in a Remote Team
Currently scraping: Collective Resilience: From Group to Team
Currently scraping: Create a Professional Online Presence
Currently scraping: Create a Social Media Marketing Campaign
Currently scraping: Decision Making: How to Choose the Right Problem to Solve
Currently scraping: Design your Purpose as a Leader
Currently scraping: Evidence and Data Collection for Problem Solving
Currently scraping: Get Creative with People to Solve Problems
Currently scraping: How to Create Great Online Content
Currently scraping: How to Deliver Results as a Team Leader
Currently scraping: Introduction to Marketing: Fundamentals of Marketing
Currently scraping: Introduction to Marketing: Omnichannel Marketing and Analysis
Currently scraping: Introduction to Marketing: Understanding your Customers
Currently scraping: Job Opportunities in Wellness
Currently scraping: Launc

Currently scraping: Microsoft Future Ready: Ethics and Law in Data and Analytics
Currently scraping: Microsoft Future Ready: Principles of Machine Learning with Python Programming
Currently scraping: Microsoft Future Ready: Using Python Programming to Explore the Principles of Machine Learning
Currently scraping: Systems Thinking for Sustainability: Analyzing and Transitioning Complex Systems
Currently scraping: Systems Thinking for Sustainability: Drawing Complexity on a Paper
Currently scraping: Systems Thinking for Sustainability: Practical Application of the SiD Method
Currently scraping: An Introduction to Macro Environmental Analysis and Business Strategy
Currently scraping: Competitor Analysis: From Business Level Strategy to Organisational Structures
Currently scraping: HR Fundamentals
Currently scraping: Internal Analysis of a Company: From Value Chain to SWOT Analysis
Currently scraping: Strategic Goals: How to Master Corporate Strategy
Currently scraping: AI and Machine Lear

Currently scraping: English for the Workplace
Currently scraping: Mergers and Acquisitions: Structuring the Deal
Currently scraping: Understanding Data in the Tourism Industry
Currently scraping: Fashion's Future: The Sustainable Development Goals
Currently scraping: Project Management Principles, Practices and Systems
Currently scraping: Challenging Wealth and Income Inequality
Currently scraping: Mergers and Acquisitions: Advanced Theory
Currently scraping: Understanding Modern Business and Organisations
Currently scraping: Introduction to Humanitarian Logistics
Currently scraping: Mergers and Acquisitions: Concepts and Theories
Currently scraping: Social Innovation in Rural Areas
Currently scraping: Introduction to Financial Accounting
Currently scraping: Introduction to the Travel & Tourism Industry: Passport to the World
Currently scraping: Digital Marketing in Perspective
Currently scraping: Digital Marketing: Content Marketing
Currently scraping: Digital Marketing: Marketing Cha

...

Currently scraping: Microsoft Future Ready: Data Science Research Methods Using Python Programming
Currently scraping: Microsoft Future Ready: Designing and Implementing an Azure AI Solution
Currently scraping: Microsoft Future Ready: Ethics and Law in Data and Analytics
Currently scraping: Microsoft Future Ready: Principles of Machine Learning with Python Programming
Currently scraping: Microsoft Future Ready: Using Python Programming to Explore the Principles of Machine Learning
Currently scraping: Systems Thinking for Sustainability: Practical Application of the SiD Method
Currently scraping: Django Features and Libraries
Currently scraping: AI Programming with Python for Beginners
Currently scraping: AI and Machine Learning Algorithms Using Python
Currently scraping: Azure Cloud Fundamentals for Data Science with Python
Currently scraping: CRM Fundamentals and Practice: Introduction to Salesforce
Currently scraping: CRM Fundamentals and Practice: Salesforce Automation and Best Prac

Currently scraping: How Computers Work: Demystifying Computation
Currently scraping: Introduction to Cybersecurity for Teachers
Currently scraping: Introduction to Databases and SQL
Currently scraping: Introduction to Machine Learning and AI
Currently scraping: Introduction to Web Development
Currently scraping: Object-oriented Programming in Python: Create Your Own Adventure Game
Currently scraping: Programming 103: Saving and Structuring Data
Currently scraping: Robotics With Raspberry Pi: Build and Program Your First Robot Buggy
Currently scraping: Start a CoderDojo Club
Currently scraping: Teaching Computing Systems and Networks to 5- to 11-year-olds
Currently scraping: Teaching Programming to 5- to 11-year-olds
Currently scraping: Get Started Teaching Computing in Primary Schools: Preparing to teach 5 - 11 year olds
Currently scraping: Networking with Python: Socket Programming for Communication
Currently scraping: Programming 102: Think Like a Computer Scientist
Currently scrapin

Currently scraping: Workplace Training: Introduction to the Training Cycle
Currently scraping: Workplace Training: Planning and Designing Training Solutions
Currently scraping: Workplace Training: Training Evaluation and Measuring Effectiveness
Currently scraping: Prepare to Run a Code Club
Currently scraping: Teach like an Entrepreneur: Bringing Entrepreneurship into the Classroom
Currently scraping: Understanding IELTS: Listening
Currently scraping: Understanding IELTS: Reading
Currently scraping: Understanding IELTS: Speaking
Currently scraping: Understanding IELTS: Writing
Currently scraping: Mindfulness: A Focus on Adolescents
Currently scraping: Learning and Memory: Understandings from Educational Neuroscience
Currently scraping: Neuroleadership and Conceptual Approaches in Educational Neuroscience
Currently scraping: Neuroplasticians and Neuromyths
Currently scraping: Orientation to Educational Neuroscience
Currently scraping: Decolonising Education: From Theory to Practice
Curr

Currently scraping: Developing and Testing Complex Healthcare Interventions
Currently scraping: English for Academic Study
Currently scraping: Digital Discovery
Currently scraping: How to Succeed at: Writing Applications
Currently scraping: Qualitative Research Methods for Mental Health in War and Conflict
Currently scraping: Study UK: Preparing for work
Currently scraping: Skills to Succeed at University
Currently scraping: Basic English 1: Elementary
Currently scraping: Basic English 2: Pre-Intermediate
Currently scraping: Community Based Research: Getting Started
Currently scraping: Critical Thinking at University: An Introduction
Currently scraping: Discovering Science: Science Writing
Currently scraping: Interdisciplinary Learning: Working Across Disciplines
Currently scraping: Introduction to Research Ethics: Working with People
Currently scraping: Learning Online: Communicating and Collaborating
Currently scraping: Learning Online: Managing Your Online Identity
Currently scrapin

Currently scraping: Understanding ADHD: Current Research and Practice
Currently scraping: Managing Mental Health and Stress
Currently scraping: Mindware: Critical Thinking for the Information Age
Currently scraping: The Science of Success: What Researchers Know that You Should Know
Currently scraping: An Introduction to the Nordoff Robbins approach to Music Therapy
Currently scraping: Caring with Music: Can Music Do More In Adult Care Settings?
Currently scraping: Supporting Adolescent Learners: Social and Emotional Wellbeing
Currently scraping: Dementia and the Arts: Sharing Practice, Developing Understanding and Enhancing Lives
Currently scraping: Emotional Intelligence at Work
Currently scraping: Identifying and Responding to Domestic Violence and Abuse (DVA) in Pregnancy
Currently scraping: Identifying and Responding to Drug and Alcohol Addiction in Nursing, Midwifery and Allied Healthcare Practice
Currently scraping: Understanding Addiction
Currently scraping: Unlocking The Creati